# 正規化層

*Normalization Layer*

入力されたデータを正規化して返す層。  
平均0, 分散1に正規化する。以後これは標準化と呼ぶ。

また本章において正規化は、標準化や0-1正規化などを含むデータのスケーリング（やシフト）を広く指す。

正規化する軸方向によって分類される。

- バッチ正規化
- 層正規化
- インスタンス正規化
- グループ正規化

pytorchで挙動を見ながら理解していこう。  
https://pytorch.org/docs/stable/nn.html#normalization-layers

In [1]:
import torch
from torch import nn


---

## バッチ正規化

*Batch Normalization*

バッチ方向に標準化を行う。  
またその後に、$d$個の特徴量ごとに対応する1次関数$f_i(\boldsymbol x_i)$を用いた変換を行う。

$$
f_i(\boldsymbol x_i)=a_ix+b_i,\quad1\leq i\leq d,\quad\boldsymbol x_i\in\mathbb R^N
$$

- $N$ : バッチサイズ

$a_i, b_i$は学習可能なパラメータ。

この変換は、平均$b_i$へのシフトと標準偏差$a_i$へのスケーリングを行っているとも見られる。  
バッチ正規化は、入力データを学習した統計量（平均, 分散）で正規化することと言える。

この層は学習時と推論時で挙動（計算方法）が変わる。

### 学習

学習時の挙動を見てみよう。  
まず適当なデータを用意する。

In [2]:
batch_size = 2
d = 5 # 特徴量の数
x = torch.arange(batch_size * d).reshape(batch_size, d).to(torch.float32)
x

tensor([[0., 1., 2., 3., 4.],
        [5., 6., 7., 8., 9.]])

これをバッチ方向に正規化する。

In [3]:
mean = x.mean(dim=0, keepdim=True)
var = x.var(dim=0, keepdim=True, unbiased=False)
eps = 1e-5 # 微小値（0除算回避）
normalized_x = (x - mean) / torch.sqrt(var + eps)
normalized_x

tensor([[-1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
        [ 1.0000,  1.0000,  1.0000,  1.0000,  1.0000]])

特徴量ごとに標準化された。

次に、これらの平均、分散を変換する。

In [4]:
a = torch.ones(d) * 2
b = torch.ones(d) * 3

y = a * normalized_x + b
y

tensor([[1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
        [5.0000, 5.0000, 5.0000, 5.0000, 5.0000]])

平均、分散は一律で$3, 2$とした。本来は特徴量ごとに異なる。

以上が学習時の挙動である。pytorchでも確認してみよう。  
https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html

In [5]:
x # 再掲

tensor([[0., 1., 2., 3., 4.],
        [5., 6., 7., 8., 9.]])

In [6]:
norm = nn.BatchNorm1d(d)
y = norm(x)
y

tensor([[-1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
        [ 1.0000,  1.0000,  1.0000,  1.0000,  1.0000]],
       grad_fn=<NativeBatchNormBackward0>)

標準化できた。  
これらの統計量ははpytorchが設定した初期値であって、学習によって変化する。

パラメータを確認してみる。

In [7]:
for params in norm.parameters():
    print(params)

Parameter containing:
tensor([1., 1., 1., 1., 1.], requires_grad=True)
Parameter containing:
tensor([0., 0., 0., 0., 0.], requires_grad=True)


特徴量の数だけパラメータ（平均, 分散）があることが分かる。

ちなみに、学習時はミニバッチ前提なので、例えばバッチサイズ1のデータを入れるとエラーが出る。

In [8]:
x[:1]

tensor([[0., 1., 2., 3., 4.]])

In [9]:
try:
    y = norm(x[:1])
except Exception as e:
    print(e)

Expected more than 1 value per channel when training, got input size torch.Size([1, 5])


### 推論

推論時は挙動が変わる。

学習時と同じようにしてしまうと、バッチ内のデータによって結果が変わってしまう。サンプリング方法によって推論結果が変わるのはマズイ。  
後は、バッチサイズ1の時も困る。学習時と分布が変わってしまうから。

推論時は、学習データ全体の統計量で正規化する。  
学習時に、特徴量ごとの統計量を記録しておく。

pytorchでみてみよう。

In [10]:
x # 再掲

tensor([[0., 1., 2., 3., 4.],
        [5., 6., 7., 8., 9.]])

In [11]:
norm = nn.BatchNorm1d(d)
norm.eval() # 推論モード
y = norm(x)
y

tensor([[0.0000, 1.0000, 2.0000, 3.0000, 4.0000],
        [5.0000, 6.0000, 7.0000, 8.0000, 9.0000]],
       grad_fn=<NativeBatchNormBackward0>)

同じデータが出力された。  
これは、パラメータに加え、学習データ全体の統計量も平均0, 分散1として初期化されているから。

In [12]:
norm.state_dict()

OrderedDict([('weight', tensor([1., 1., 1., 1., 1.])),
             ('bias', tensor([0., 0., 0., 0., 0.])),
             ('running_mean', tensor([0., 0., 0., 0., 0.])),
             ('running_var', tensor([1., 1., 1., 1., 1.])),
             ('num_batches_tracked', tensor(0))])

`running_mean`と`running_var`が学習データ全体の統計量。  
これらは学習モードで演算を行うと勝手に更新される。

In [13]:
norm.train()
y = norm(x)
norm.state_dict()

OrderedDict([('weight', tensor([1., 1., 1., 1., 1.])),
             ('bias', tensor([0., 0., 0., 0., 0.])),
             ('running_mean',
              tensor([0.2500, 0.3500, 0.4500, 0.5500, 0.6500])),
             ('running_var', tensor([2.1500, 2.1500, 2.1500, 2.1500, 2.1500])),
             ('num_batches_tracked', tensor(1))])

In [14]:
print('mean:', x.mean(dim=0))
print('var:', x.var(dim=0))

mean: tensor([2.5000, 3.5000, 4.5000, 5.5000, 6.5000])
var: tensor([12.5000, 12.5000, 12.5000, 12.5000, 12.5000])


入力したデータの平均と分散が記録された。  
`momentum=0.1`がデフォルトで設定されているので、完全には一致していない。

これでもう一度推論モードにしてみると、今記録した統計量で正規化が行われる。

In [15]:
norm.eval()
y = norm(x)
y

tensor([[-0.1705,  0.4433,  1.0571,  1.6709,  2.2847],
        [ 3.2395,  3.8533,  4.4671,  5.0808,  5.6946]],
       grad_fn=<NativeBatchNormBackward0>)

確かめてみよう。

In [16]:
mean = norm.state_dict()['bias'] # 平均（パラメータ）
var = norm.state_dict()['weight'] # 分散（パラメータ）
data_mean = norm.state_dict()['running_mean'] # 平均（学習データ）
data_var = norm.state_dict()['running_var'] # 分散（学習データ）
eps = norm.eps

((x - data_mean) / torch.sqrt(data_var + eps)) * var + mean

tensor([[-0.1705,  0.4433,  1.0571,  1.6709,  2.2847],
        [ 3.2395,  3.8533,  4.4671,  5.0808,  5.6946]])

ちゃんと同じになったね。

あと、バッチサイズ1の時もちゃんと動く。

In [17]:
norm.eval()
norm(x[:1])

tensor([[-0.1705,  0.4433,  1.0571,  1.6709,  2.2847]],
       grad_fn=<NativeBatchNormBackward0>)

特徴量は2次元でもいい。まとめて正規化される。

In [18]:
batch_size = 2
h = 3
w = 4
x = torch.arange(batch_size * h * w).reshape(batch_size, h, w).to(torch.float32)
x

tensor([[[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.]],

        [[12., 13., 14., 15.],
         [16., 17., 18., 19.],
         [20., 21., 22., 23.]]])

In [19]:
norm = nn.BatchNorm1d(h)
norm(x)

tensor([[[-1.2288, -1.0650, -0.9012, -0.7373],
         [-1.2288, -1.0650, -0.9012, -0.7373],
         [-1.2288, -1.0650, -0.9012, -0.7373]],

        [[ 0.7373,  0.9012,  1.0650,  1.2288],
         [ 0.7373,  0.9012,  1.0650,  1.2288],
         [ 0.7373,  0.9012,  1.0650,  1.2288]]],
       grad_fn=<NativeBatchNormBackward0>)

こういうこと。

In [20]:
mean = x.mean(dim=(0, 2), keepdim=True)
mean

tensor([[[ 7.5000],
         [11.5000],
         [15.5000]]])

In [21]:
var = x.var(dim=(0, 2), keepdim=True, unbiased=False)
var

tensor([[[37.2500],
         [37.2500],
         [37.2500]]])

In [22]:
((x - mean) / torch.sqrt(var + norm.eps))

tensor([[[-1.2288, -1.0650, -0.9012, -0.7373],
         [-1.2288, -1.0650, -0.9012, -0.7373],
         [-1.2288, -1.0650, -0.9012, -0.7373]],

        [[ 0.7373,  0.9012,  1.0650,  1.2288],
         [ 0.7373,  0.9012,  1.0650,  1.2288],
         [ 0.7373,  0.9012,  1.0650,  1.2288]]])

バッチ次元（0番目）の次の次元（1番目）が単位となってまとまるので、それ以外の軸を指定して（`dim=(0, 2)`）計算するというイメージ。

ちなみに3次元以上の特徴量（4次元以上の入力）には対応していない。

In [23]:
batch_size = 2
c = 3
w = 4
h = 5
x = torch.randn(batch_size, c, w, h)

norm = nn.BatchNorm1d(c)
try:
    norm(x)
except Exception as e:
    print(e)

expected 2D or 3D input (got 4D input)


特徴量が3次元の場合は別のクラスを使う。  
https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html

CNNで使われる事が多い。特徴マップの正規化。

In [24]:
batch_size = 2
c = 3
w = 4
h = 5
x = torch.randn(batch_size, c, w, h)
x.shape

torch.Size([2, 3, 4, 5])

In [25]:
norm = nn.BatchNorm2d(c)
for params in norm.parameters():
    print(params)

Parameter containing:
tensor([1., 1., 1.], requires_grad=True)
Parameter containing:
tensor([0., 0., 0.], requires_grad=True)


In [26]:
y = norm(x)
y.shape

torch.Size([2, 3, 4, 5])

後ろの二つの次元をまとめれば`BatchNorm1d`でも同じことが出来る。

In [27]:
norm = nn.BatchNorm1d(c)
y_ = norm(x.reshape(batch_size, c, -1)).reshape(batch_size, c, w, h)
torch.equal(y, y_)

True

ちなみに4次元以上の特徴量を扱う`BatchNorm3d`もある。  
https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm3d.html

特徴量の数と対応しているclassをまとめるとこうなる。

| 特徴量の次元数 | 入力の次元数 | class |
| --- | --- | --- |
| 1 | 2 | `BatchNorm1d` |
| 2 | 3 | `BatchNorm1d` |
| 3 | 4 | `BatchNorm2d` |
| 4 | 5 | `BatchNorm3d` |

命名と分け方が謎だ。  
何で特徴量の次元数と`{n}d`が一致しないんだろう。何で`1d`が二つ分担当しているんだろう。


---

## 層正規化

*Layer Normalization*

層方向に標準化を行う。層方向というのは、特徴量方向って感じ。  
バッチ正規化と同じように、標準化した後に特徴量ごとにスケーリングとシフトを行う。

標準化はバッチ単位、スケーリングとシフトは特徴量単位なので、バッチ正規化のように全体（標準化→シフト→スケーリング）を正規化と捉えることは出来ないと思う。でも面倒くさいので全体の演算も正規化と呼ぶことにする。

あと層正規化は演算結果がバッチ内の他のデータに依らないので、学習時と推論時で挙動が変わらない。

RNNにバッチ正規化の適用が困難（系列長が固定でないから）なことから生まれた、多分。

では適当にやってみよう。

In [28]:
batch_size = 2
d = 5
x = torch.arange(batch_size * d).reshape(batch_size, d).to(torch.float32)
x

tensor([[0., 1., 2., 3., 4.],
        [5., 6., 7., 8., 9.]])

層作成。  
特徴量の形状を与える。その形状と同じ平均と分散がパラメータとなる。

In [29]:
norm = nn.LayerNorm(d)
for params in norm.parameters():
    print(params)

Parameter containing:
tensor([1., 1., 1., 1., 1.], requires_grad=True)
Parameter containing:
tensor([0., 0., 0., 0., 0.], requires_grad=True)


学習データ全体の統計量は持っていない。

In [30]:
norm.state_dict()

OrderedDict([('weight', tensor([1., 1., 1., 1., 1.])),
             ('bias', tensor([0., 0., 0., 0., 0.]))])

正規化。

In [31]:
y = norm(x)
y

tensor([[-1.4142e+00, -7.0710e-01,  0.0000e+00,  7.0710e-01,  1.4142e+00],
        [-1.4142e+00, -7.0710e-01,  1.7881e-07,  7.0711e-01,  1.4142e+00]],
       grad_fn=<NativeLayerNormBackward0>)

以下と同じ。

In [32]:
mean = x.mean(dim=-1, keepdim=True)
var = x.var(dim=-1, keepdim=True, unbiased=False)
((x - mean) / torch.sqrt(var + norm.eps))

tensor([[-1.4142, -0.7071,  0.0000,  0.7071,  1.4142],
        [-1.4142, -0.7071,  0.0000,  0.7071,  1.4142]])

統計量を求める軸を層（特徴量）方向に指定した（`dim=-1`）。

特徴量は何次元でもいい。例えばRNNの場合、以下のようにすれば各バッチ各隠れ状態が正規化される。

In [33]:
batch_size = 2
seq_len = 3
d = 4
x = torch.arange(batch_size * seq_len * d).reshape(batch_size, seq_len, d).to(torch.float32)
x

tensor([[[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.]],

        [[12., 13., 14., 15.],
         [16., 17., 18., 19.],
         [20., 21., 22., 23.]]])

In [34]:
norm = nn.LayerNorm(d)
for params in norm.parameters():
    print(params)

Parameter containing:
tensor([1., 1., 1., 1.], requires_grad=True)
Parameter containing:
tensor([0., 0., 0., 0.], requires_grad=True)


In [35]:
y = norm(x)
y

tensor([[[-1.3416, -0.4472,  0.4472,  1.3416],
         [-1.3416, -0.4472,  0.4472,  1.3416],
         [-1.3416, -0.4472,  0.4472,  1.3416]],

        [[-1.3416, -0.4472,  0.4472,  1.3416],
         [-1.3416, -0.4472,  0.4472,  1.3416],
         [-1.3416, -0.4472,  0.4472,  1.3416]]],
       grad_fn=<NativeLayerNormBackward0>)

In [36]:
mean = x.mean(dim=-1, keepdim=True)
var = x.var(dim=-1, keepdim=True, unbiased=False)
((x - mean) / torch.sqrt(var + norm.eps))

tensor([[[-1.3416, -0.4472,  0.4472,  1.3416],
         [-1.3416, -0.4472,  0.4472,  1.3416],
         [-1.3416, -0.4472,  0.4472,  1.3416]],

        [[-1.3416, -0.4472,  0.4472,  1.3416],
         [-1.3416, -0.4472,  0.4472,  1.3416],
         [-1.3416, -0.4472,  0.4472,  1.3416]]])